<a href="https://colab.research.google.com/github/michalis0/DataMining_and_MachineLearning/blob/master/week2/Data_Cleaning_in_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data cleaning with Pandas

<img width="200" src='https://upload.wikimedia.org/wikipedia/commons/e/ed/Pandas_logo.svg'>

### How Does Data Get Dirty?
- Missing data.
- Inconsistent data.
- Duplicate data.

This is just to name a few things that can go wrong. There is an endless list of ways that data can end up very messy.
Sometimes there are insufficient validation checks when the data is entered in the first place.
If you have form fields with users entering data in any format they want with no guidelines or form validation checks in place to enforce conforming to a certain format, then users will input however they see fit.

There could be an input field for the state (U.S.) and you have some data that is the two-character abbreviation, NY and then others have New York, then there are potential misspellings and typos, etc.

Data can also become corrupted during transmission or in storage.

We’ll use Pandas to examine and clean the building violations dataset from the NYC Department of Buildings (DOB) that is available on NYC Open Data.

The datset can be found [here](https://data.cityofnewyork.us/Housing-Development/DOB-Violations/3h2n-5cm9).

For this exercise, we will work with a subset of the data with 10000 records.

In [1]:
import pandas as pd 
import numpy as np 

### Load the data

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/michalis0/DataMining_and_MachineLearning/master/week3/data/violation_DOB.csv")

### Shape of the data
Let's first check how many rows and columns (features) are in this dataset

In [3]:
df.shape

(10000, 18)

### Check out the first few rows
You can look at the first few rows by calling `head()` on the dataframe.

In [4]:
df.head(3)

,isn_dob_bis_viol,boro,bin,block,lot,issue_date,violation_type_code,violation_number,house_number,street,device_number,description,number,violation_category,violation_type,disposition_date,disposition_comments,ecb_number
0,2351406,3,3059563,2136.0,2136.0,20190520,JVCAT5,00339,441,WYTHE AVENUE,3P10799,FAILURE TO PERFORM CATEGORY 5 INSPECTION,V052019JVCAT500339,V-DOB VIOLATION - ACTIVE,JVCAT5-RESIDENTIAL ELEVATOR PERIODIC INSPECTIO...,NaN,NaN,NaN
1,2383173,3,3137310,5631.0,14.0,20190903,LL2604S,NRF01042,920,48 STREET,NaN,FAILED TO FILE FINAL SPRINKLER REPORT BY JULY ...,V090319LL2604SNRF01042,V-DOB VIOLATION - ACTIVE,LL2604S-SPRINKLER,NaN,NaN,NaN
2,2427322,2,2003035,2504.0,36.0,20190904,AEUHAZ1,00162,941,JEROME AVENUE,NaN,FAILURE TO CERTIFY CORRECTION ON IMMEDIATELY H...,V*090419AEUHAZ100162,V*-DOB VIOLATION - DISMISSED,AEUHAZ1-FAIL TO CERTIFY CLASS 1,20191202.0,GNC PAID INVOICE 62132098,35409075X


### List the column/feature names

In [5]:
df.columns

Index(['isn_dob_bis_viol', 'boro', 'bin', 'block', 'lot', 'issue_date',
       'violation_type_code', 'violation_number', 'house_number', 'street',
       'device_number', 'description', 'number', 'violation_category',
       'violation_type', 'disposition_date', 'disposition_comments',
       'ecb_number'],
      dtype='object')

### Duplicates
You can check if there are duplicates in the dataset.

In [6]:
df.duplicated()  # this gives a Boolean data type

0       False
1       False
2       False
3       False
4       False
        ...  
9995    False
9996    False
9997    False
9998    False
9999    False
Length: 10000, dtype: bool

This shows which are the duplicated rows if any. None are duplicated here.

In [7]:
df.loc[df.duplicated()=='True', :]

,isn_dob_bis_viol,boro,bin,block,lot,issue_date,violation_type_code,violation_number,house_number,street,device_number,description,number,violation_category,violation_type,disposition_date,disposition_comments,ecb_number


For example you can check that if you only look at house numbers, then the same house appears more than once in the dataset. Looking at house numbers only the dataset does contain duplicates.

In [8]:
df.house_number.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
9995     True
9996     True
9997     True
9998     True
9999     True
Name: house_number, Length: 10000, dtype: bool

### Missing/Null values
You can call `isnull()` and `sum()` to get a count of how many null values are there in each column.

In [9]:
df.isnull().sum()

isn_dob_bis_viol           0
boro                       0
bin                        0
block                      1
lot                        1
issue_date                 0
violation_type_code        0
violation_number           0
house_number               0
street                     0
device_number           4775
description              528
number                     0
violation_category         0
violation_type             0
disposition_date        6297
disposition_comments    6285
ecb_number              7532
dtype: int64

### Data Types of the Features
It is important that the data values in each column have the correct data type. For example, you can expect a column containing numbers to be in numeric format, but sometimes you will find string values in it. In such a case, when you do numeric calculations on that column you might get unexpected results.

The property `dtypes` will show you the data types for each column in the dataframe.

In [10]:
df.dtypes

isn_dob_bis_viol          int64
boro                     object
bin                       int64
block                   float64
lot                     float64
issue_date                int64
violation_type_code      object
violation_number         object
house_number             object
street                   object
device_number            object
description              object
number                   object
violation_category       object
violation_type           object
disposition_date        float64
disposition_comments     object
ecb_number               object
dtype: object

You can notice that the column `issue_date` is in integer format (`int64`), whereas it should be in datetime format. In Pandas you can convert a column to datetime format using `to_datetime` method.

In [11]:
df['issue_date'].head()

0    20190520
1    20190903
2    20190904
3    20190906
4    20190107
Name: issue_date, dtype: int64

In [12]:
# if you do not provide the format, the function may infer the wrong format
pd.to_datetime(df['issue_date']).head()


0   1970-01-01 00:00:00.020190520
1   1970-01-01 00:00:00.020190903
2   1970-01-01 00:00:00.020190904
3   1970-01-01 00:00:00.020190906
4   1970-01-01 00:00:00.020190107
Name: issue_date, dtype: datetime64[ns]

In [13]:
df['issue_date'] = pd.to_datetime(df['issue_date'], format="%Y%m%d")
df['issue_date'].head()

0   2019-05-20
1   2019-09-03
2   2019-09-04
3   2019-09-06
4   2019-01-07
Name: issue_date, dtype: datetime64[ns]

As another example you can also convert the `house_number` column to numeric datatype. To do so you can use the `to_numeric` method.

>Note that in a real application you should not do that since you are not going to do numeric calculations on `house_number`. Moreover `house_number` could be a string or a mix of string and digits as well).



In [14]:
# notice that by running this line you will get an error. There reason is that there are non-numeric values in this
# column which cannot be converted to numeric data type. 
df["house_number"] = pd.to_numeric(df['house_number'])

ValueError: ignored

In [15]:
# However, you can deal with such errors by ignoring them and replacing them with NaN. To do so
# you should set errors='coerce'
df["house_number"] = pd.to_numeric(df['house_number'], errors='coerce')

In [16]:
df.dtypes

isn_dob_bis_viol                 int64
boro                            object
bin                              int64
block                          float64
lot                            float64
issue_date              datetime64[ns]
violation_type_code             object
violation_number                object
house_number                   float64
street                          object
device_number                   object
description                     object
number                          object
violation_category              object
violation_type                  object
disposition_date               float64
disposition_comments            object
ecb_number                      object
dtype: object

The coerce parameter used above generated new missing values for this `house_number` column.

In [17]:
df.isnull().sum()

isn_dob_bis_viol           0
boro                       0
bin                        0
block                      1
lot                        1
issue_date                 0
violation_type_code        0
violation_number           0
house_number            1740
street                     0
device_number           4775
description              528
number                     0
violation_category         0
violation_type             0
disposition_date        6297
disposition_comments    6285
ecb_number              7532
dtype: int64

### Dropping the columns you are not interested in
Let's say we are only interested in house number, the types of violations each building received and whether they have been closed or not. So we are going to drop the rest of the columns.

Note that many operations in Pandas can be done on a copy of the data or in-place (affecting the original data frame). You can use the parameter `inplace=True`. 

> Also note that in Pandas, axis 0 represents the rows while axis 1 represents the columns.

In [18]:
columns_to_delete = ['block', 'boro','lot','street','violation_number',
                     'disposition_comments', 'isn_dob_bis_viol', 'disposition_date','ecb_number','description'] 
df.drop(columns_to_delete, inplace=True, axis=1)

In [19]:
df.columns

Index(['bin', 'issue_date', 'violation_type_code', 'house_number',
       'device_number', 'number', 'violation_category', 'violation_type'],
      dtype='object')

### Dealing with null values
There are different ways to deal with null values in a dataset.   
Here we are going to show you two different cases.

1. Filling null values with the mean or median value of the column (for numerical features).

2. Removing the rows (data samples) with null features.


In [20]:
# replacing null values in the house_number column with the mean value
# note that you can also replace the null values with any other scalar as well
df["house_number"].fillna(df["house_number"].mean(), inplace=True)

In [21]:
# Removing the rows (data samples) with null features
df.dropna(axis=0, inplace=True)

In [22]:
# now we have less rows in our dataframe
df.shape

(5225, 8)

### Categorical features
In pandas there is also a specific data type for categorical features. Categorical features are the ones that only take values from a given set. For instance in this dataset `violation_type_code` is a catgorical feature.

You can convert a column to categorical using the method `astype('category')`.

In [23]:
df.dtypes

bin                             int64
issue_date             datetime64[ns]
violation_type_code            object
house_number                  float64
device_number                  object
number                         object
violation_category             object
violation_type                 object
dtype: object

With the help of the `value_counts()` function you can see how many times each violation code is present in the dataset. This is a categorical nominal variable.

In [24]:
df["violation_type_code"].value_counts()

LBLVIO     1946
ACC1       1393
EVCAT1      676
EVCAT5      476
FISPNRF     221
E           201
FISP         97
JVIOS        70
JVCAT5       68
ACH1         37
ACJ1         16
HVIOS        15
HVCAT5        9
Name: violation_type_code, dtype: int64

In [25]:
df["violation_type_code"] = df["violation_type_code"].astype("category")

You can see that now the data type of this column has changed. Also observe that we have 13 categories in the column.

In [26]:
df["violation_type_code"]

0       JVCAT5
3         ACC1
4            E
7       LBLVIO
8       EVCAT5
         ...  
9994    LBLVIO
9996    LBLVIO
9997    LBLVIO
9998    EVCAT1
9999    LBLVIO
Name: violation_type_code, Length: 5225, dtype: category
Categories (13, object): ['ACC1', 'ACH1', 'ACJ1', 'E', ..., 'HVIOS', 'JVCAT5', 'JVIOS', 'LBLVIO']

In [27]:
df.violation_type_code.cat.categories

Index(['ACC1', 'ACH1', 'ACJ1', 'E', 'EVCAT1', 'EVCAT5', 'FISP', 'FISPNRF',
       'HVCAT5', 'HVIOS', 'JVCAT5', 'JVIOS', 'LBLVIO'],
      dtype='object')

### Change a column name

You can change columns names easily in pandas. To do so, use the `rename` method and pass the names to change as a dictionary.

In [28]:
df.rename(columns = {"issue_date": "date"}, inplace=True)

In [29]:
df.head(2)

,bin,date,violation_type_code,house_number,device_number,number,violation_category,violation_type
0,3059563,2019-05-20,JVCAT5,441.0,3P10799,V052019JVCAT500339,V-DOB VIOLATION - ACTIVE,JVCAT5-RESIDENTIAL ELEVATOR PERIODIC INSPECTIO...
3,1001389,2019-09-06,ACC1,375.0,1F5381,V090619ACC100284,V-DOB VIOLATION - ACTIVE,ACC1-(OTHER BLDGS TYPES) - ELEVATOR AFFIRMATIO...
